In [55]:
import random
import numpy as np
from keras.datasets import mnist

In [56]:
X_train_path = './X_train'
Y_train_path = './Y_train'
X_test_path = './X_test'

In [111]:
class Network(object):
    
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedfoward(self, a):
        
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        
        x, y = zip(*train)
        x_test, y_test = zip(*test)
        if test_data: 
            n_test = len(x_test)
        n = len(x)
        for j in range(epochs):
            mini_batches_x = [
                x[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            mini_batches_y = [
                y[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch_x, mini_batch_y in zip(mini_batches_x, mini_batches_y):
                self.update_mini_batch(mini_batch_x, mini_batch_y, eta)
            if test_data:
                print('Epoch %d: %d / %d' % (j ,self.evaluate(test_data, n_test)))
            else:
                print('Epoch %d complete' % j)
                
    def update_mini_batch(self, mini_batch_x, mini_batch_y, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(mini_batch_x, mini_batch_y):
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biase = [b - (eta / len(mini_batch)) * nb
                      for b, nb in zip(self.biases, nabla_b)]
        
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-1]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-1] = delta
            nabla_w[-1] = np.dot(delta, activations[-l-1].transpose())
        
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedfoward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_actovations, y):
        return (output_actovations - y)
    
def sigmoid(z):
    return 1. / (1. + np.exp(-z))
    
def sigmoid_prime(z):
    return sigmoid(z) * (1-sigmoid(z))

In [112]:
def one_hot(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

def load_data():
    train, test = mnist.load_data()
    x = train[0]
    y = [one_hot(y_) for y_ in train[1]]
    x_test = test[0]
    y_test = [one_hot(y_) for y_ in test[1]]
    x = x.reshape(-1, 784)
    x_test = x_test.reshape(-1, 784)
    return zip(x, y), zip(x_test, y_test)
    
train, test = load_data()
net = Network([784, 30, 10])
net.SGD(train, 30, 10, 3.0, test)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:81: RuntimeWarning: overflow encountered in exp


ValueError: operands could not be broadcast together with shapes (30,30) (30,10) 